## Hypothesis testing for not normal populations

En este caso donde no podemos asumir que las muestras bajo análisis provengan de una población normal. Lo que tenemos que analizar en estos casos son las medianas de ambos grupos. Estos test de hipótesis se suelen decir que son no parámetros dado que no se basan en una distribución en particual sino que dependiendo el test deberemos utilizar distintas herramientas para representarlos.

### One sample sign test (test binomial)

En estos casos queremos comprar la mediana de una muestra vs una mediana objetivo. En estos casos, lo único que nos interesa analizar es si la muestra es aleatoria y sus observaciones son independientes.

El **estadístico** que vamos a utilizar en este caso es: El mínimo de la cantidad valores positívos o valores negativos. Los valores se representan como una variable aleatoria que muestra las diferencias entre la muestra y la mediana objetivo. Pero una vez que tenemos esto tenemos que tener una forma también de calcular el p-value de ese estadístico o el valor crítico correspondiente a un determinado alpha. Pero ¿Cómo lo hacemos si ahora no tenemos una distribución dada?

#### ¿Cómo calculamos le p-value?
En estos casos nosotros deberíamos poder asignarle una distribución a este tipo de problema. Cómo nosotros estamos trabajando con medianas, es decir, con un percentil del 50%, en la población objetivo nosotros deberíamos poder observar igual cantidad de diferencias positivas y negativas si es que la mediana es exactamente la que nosotros estamos contrastando. Con lo cual esto se vuelve un experimiento dicotómico. 

Cuales son las características de la binomial:

* Tenemos un experimiento con n cantidad de intentos
* Cada uno de los intentos es independiente del otro
* La probabilidad de éxito es constante a lo largo del experimiento
* Solo tenemos dos alternativas


#### Ejemplo
En el caso que quisieramos contrastar una muestra que representa el tiempo de resolución de un problema a través de la atención telefónica con una mediana hipotética de 15 minutos. Tenemos que...

In [1]:
# Importamos las librerías
import numpy as np
import pandas as pd
import scipy.stats as st

# Creamos una muestra aleatoria
muestra = pd.DataFrame(data = {'muestra': np.random.uniform(low=10, high=22, size=15).round(0)})

# Mediana a contrastar y alpha
mediana_ho = 15
alpha = 0.05

In [2]:
muestra['diferencia'] = muestra['muestra'] - mediana_ho
muestra.head(5)

,muestra,diferencia
0,21.0,6.0
1,13.0,-2.0
2,14.0,-1.0
3,11.0,-4.0
4,12.0,-3.0


In [3]:
# Eliminamos los valores cuya diferencia sea negativa
muestra = muestra[muestra['diferencia'] != 0]
muestra.shape

(15, 2)

In [4]:
# Aplicamos la función signo
muestra['signo'] = muestra['diferencia'].apply(lambda x: 1 if x > 0 else 0)
muestra.head(10)

,muestra,diferencia,signo
0,21.0,6.0,1
1,13.0,-2.0,0
2,14.0,-1.0,0
3,11.0,-4.0,0
4,12.0,-3.0,0
5,19.0,4.0,1
6,14.0,-1.0,0
7,18.0,3.0,1
8,14.0,-1.0,0
9,12.0,-3.0,0


In [5]:
# Sacamos los valores positivos y negativos y definimos el estadístico
negativo = muestra['signo'].value_counts()[0]
positivo = muestra['signo'].value_counts()[1]

# Definimos el estadístico
stat = min(negativo, positivo)
stat

np.int64(6)

In [6]:
# Ahora lo que hacemos es buscar el valor crítico para la primera cola
critival_1 = st.binom.ppf(0.025, n=(negativo+positivo), p=0.5)

# Ahora lo que hacemos es buscar el valor crítico para la segunda cola
critival_2 = st.binom.ppf(0.975, n=(negativo+positivo), p=0.5)

# Calculamos el p-value para nuestr estadístico pero para el caso de ambas colas
p_value = 2 * st.binom.cdf(stat, n=(negativo+positivo), p=0.5)
print(p_value)

0.6072387695312499


In [7]:
# En python, esto lo podemos hacer con el test binomial
p_value_dos = st.binomtest(stat, n=(negativo+positivo), p=0.5, alternative='two-sided')
p_value_dos

BinomTestResult(k=6, n=15, alternative='two-sided', statistic=0.4, pvalue=0.6072387695312499)

In [8]:
# Marcamos la conclusión
print("La conclusión es que como el p-value no esta dentro de la zona de rechazo, no podemos rechzar la hipótesis nula y no existen evidencias significativas \
para pensar que la mediana de la población es distinta.")

La conclusión es que como el p-value no esta dentro de la zona de rechazo, no podemos rechzar la hipótesis nula y no existen evidencias significativas para pensar que la mediana de la población es distinta.


### Wilcoxon signed rank test (para muestras pequeñas!)

Este test a diferencia del primero sirve para comprar dos muestras pero que en este caso son dependientes entre sí o están pareadas. Nuevamente lo que vamos a hacer es comparar ambas muestras pero alrededor de sus medianas. Los supuestos que tenemos que tener en cuenta son:

* Las muestras tienen que haberse elegido al azar.
* Dentro de cada muestra las observaciones tiene que ser independientes.
* La distribución de diferencia entre muestras tiene que ser simétrica.

El **estadístico** que vamos a utilizar en este caso es el mínimo valor entre la suma del rank de las diferencias positivas(t+) y la suma del rank de las diferencias negativas (t-).

#### ¿Que sucede con los estadísticos a 1 cola?

**Ho: antes.median() = despues.median()** \
**Ha: antes.median() < despues.median()**

En el caso que querramos comprabar, por ejemplo, que la mediana de las ventas luego del tratmiento es mayor a la mediana antes del tramiento, en este caso tendremos que las diferencias negativas (t-) van a ser mayores que las diferencias positivas (t+) y eso es un indicio de un aumento general en las muestras luego del tratmiento (esto se da porque la diferencia que calculamos es antes-despues). Por esto es que vamos a comprar el valor crítico contra (t+). Si el valor de (t+) es menor o igual al valor crítico vamos a rechazar la hipótesis nula.

#### Ejemplo
En este caso, supongamos que una empresa esta sacando al mercado una nueva oferta sobre un determinado servicio. Lo que queremos hacer es evaluar una muestra de clientes antes y después de haber recibido la oferta para ver si hay un cambio significativo en las ventas.

En este caso como sabemos que estamos haciendo un experimento de antes y después sabemos que tenemos muestras que son dependientes.

**Ho: antes.median() = despues.median()** \
**Ha: antes.median() != despues.median()**

In [9]:
# Creamos el dataframe
data = {'antes': [233,388,351,348,376,385,368,266,283,313,376,376],
       'despues': [453,257,501,386,274,385,283,375,433,415,158,300]}
df = pd.DataFrame(data)

In [10]:
# calculamos las diferencias y descartamos las que son nulas
df['diferencia'] = df['antes'] - df['despues']

# Eliminamos los valores cuya diferencia sea cero
df = df[df['diferencia'] != 0]
df.shape

(11, 3)

In [11]:
# Eliminamos los valores cuya diferencia sea cero
df['abs'] = df['diferencia'].abs()
df.head(5)

,antes,despues,diferencia,abs
0,233,453,-220,220
1,388,257,131,131
2,351,501,-150,150
3,348,386,-38,38
4,376,274,102,102


In [12]:
# Calcualmos el ranking (lo bueno de esto es que ya calcula el ranking promedio para los datos que coinciden)
df['Ranking'] = df['abs'].rank(ascending=True)
df.head(6)

,antes,despues,diferencia,abs,Ranking
0,233,453,-220,220,11.0
1,388,257,131,131,7.0
2,351,501,-150,150,8.5
3,348,386,-38,38,1.0
4,376,274,102,102,4.5
6,368,283,85,85,3.0


In [13]:
# Ahora lo que hacemos es sumar los ranking que tienen una diferencia negativo y los que tiene una diferencia positiva por otro
sum_rank_neg = df[df['diferencia'] < 0]['Ranking'].sum()
sum_rank_pos = df[df['diferencia'] > 0]['Ranking'].sum()

# imprimimos ambos
print(sum_rank_neg)
print(sum_rank_pos)

# Calculamos también nuestro W_statistic
W_statistic = min(sum_rank_neg, sum_rank_pos)

# Para luego ver el estadístico tenemos que ir a la tabla de forma manual
# https://users.stat.ufl.edu/~winner/tables/wilcox_signrank.pdf
# Ahí podemos ver que nuestro W_critico es 10

39.5
26.5


In [14]:
# Conclusión:
# En este caso, como nuestro estadístico es mayor que el valor crítico de 10 no podemos rechazar la hipótesis nula y no podemos inferir que el cambio en 
# la política haya mejorado las ventas. 

In [15]:
# Podemos hacer esto mismo en python
st.wilcoxon(df['antes'], df['despues'], alternative = 'two-sided', nan_policy="omit")

WilcoxonResult(statistic=np.float64(26.5), pvalue=np.float64(0.6376953125))

### Mann-Whitney U-test

Este test es el paralelo al two sample T-test pero en este caso lo que queremos comprobar es si provienen de poblaciones con medianas diferentes. La idea es la misma que antes, si ambas muestras provienen de la misma población si las agrupamos y las ordenamos de menor a mayor deberian estas observaciones intercalarse aleatoriamente, si vemos que por ejemplo, todos los valores de una población se agrupan a la derecha quiere decir que la mediana de dicha población es mayor a la otra.

Si esto lo pensamos como diferencias deberíamos ver que si ambas vienen de la misma población esas diferencias las diferencias positivas como las diferencia negativas deberían ser las mismas. Por lo tanto ambas medianas poblacionales deberían ser iguales.

Como el U-Test es un test no paramétrico puede ser menos potente que el t-test (tiene menos probabilidad de rechazar la hipótesis nula cuando verdaderamente es falsa) puesto que no diferencia valores extremos productos de los rankings que hace. Pero esto hace que el U-test sea más robusto que el t-test.

Las condiciones que se tienen que cumplir son:

* Las muestras tienen que haberse elegido al azar.
* Dentro de cada muestra las observaciones tiene que ser independientes y también tiene que ocurrrir entre ambas muestras
* La distribución de ambas muestras tiene que tener la misma forma (simetría, dispersión, kurtosis, etc.) para que podamos utilizar las medianas. Algo que podemos utilizar para esto es ver también que sean homocedaśticas por ejemplo

El **estadístico** es similar al caso anterior: el mínimo entre la suma de rankings pero ajustada por el temaño de la muestra como se verá en el ejemplo.

#### ¿Qué potenciales problemas puedo encontrar?

Si las distribuciones poblacionales subyacentes de cada muestra solo se diferencia en la localización entonces si podemos decir que el U-Test compara medianas. Pero eso es poco realiasta y no suele ocurrir. Es en estos casos donde la interpretación de los p-values suele ser más compleja que en el T-Test dado que lo que estamos contrastando son probabilidades. Por ejemplo (citando a Joaquín Amat): "Si las dos muestras comparadas proceden de poblaciones con asimetrías en direcciones opuestas, a pesar de que tanto la media como la mediana sean exactamente las mismas en ambos grupos, el p-value obtenido puede ser muy bajo. Cuando la hipótesis que se quiere contrastar tiene que ser exclusivamente la igualdad de medianas se puede recurrir a la regresión de cuantiles, en concreto al cuantil 0.5, o a los test de permutaciones o bootstrapping. "

Por otro lado, a medida que el tamaño muestral aumenta el U-Test es muy sensible detectando diferencias en las distribuciones poblacionales a pesar que sus medianas sean similares.


#### ¿Qué es lo que de verdad estamos constratando?

Por ejemplo, si estamos analizando dos muestras que tienen asimetrías opuestas podemos tener, aún así, medianas que son las mismas pero obtener un p-value muy bajo que nos haga contrastar la hipótesis nula de igualdad de medianas. El problema es que lo que verdaderamente nos termina diciendo este test es que existen evidencias para contrastar que las observaciones de una muestra tienen mayor probabilidad de estar por encima de la otra.

#### Ejemplo


**Ho: antes.median() = despues.median()** \
**Ha: antes.median() != despues.median()**

In [16]:
# Creamos el dataframe
muestra_1 = {'muestra_1': [16,11,15,19,20,20,11,20]}
muestra_2 = {'muestra_2': [13,25,25,13,16,17,18,13,25,16]}

df = pd.DataFrame(muestra_1)
df_2 = pd.DataFrame(muestra_2)

df['grupo'] = 'group_1'
df_2['grupo'] = 'group_2'

# Data concatenado
df_concat = pd.concat([df, df_2]).reset_index(drop=True)
df_concat['observaciones'] = np.where(df_concat['muestra_1'].isnull(), df_concat['muestra_2'], df_concat['muestra_1'])
df_concat.drop(columns = ['muestra_1', 'muestra_2'], inplace=True)
df_concat

,grupo,observaciones
0,group_1,16.0
1,group_1,11.0
2,group_1,15.0
3,group_1,19.0
4,group_1,20.0
5,group_1,20.0
6,group_1,11.0
7,group_1,20.0
8,group_2,13.0
9,group_2,25.0


In [17]:
# Ahora lo que hacemos es calcular el ranking de todas las observaciones
df_concat['Ranking'] = df_concat['observaciones'].rank(ascending=True)
df_concat

,grupo,observaciones,Ranking
0,group_1,16.0,8.0
1,group_1,11.0,1.5
2,group_1,15.0,6.0
3,group_1,19.0,12.0
4,group_1,20.0,14.0
5,group_1,20.0,14.0
6,group_1,11.0,1.5
7,group_1,20.0,14.0
8,group_2,13.0,4.0
9,group_2,25.0,17.0


In [18]:
# AHora lo que hacemos es calcular la suma de los rankings para cada grupos
sum_rank_1 = df_concat[df_concat['grupo'] == 'group_1']['Ranking'].sum()
sum_rank_2 = df_concat[df_concat['grupo'] == 'group_2']['Ranking'].sum()

# imprimimos ambos
print(sum_rank_1)
print(sum_rank_2)

# Ahora calculamos el estadístico pero teniendo en cuenta este supuesto de que las distribuciones poblaciones son similares
u_1 = sum_rank_1 - (df.shape[0]*(df.shape[0]+1))/2
u_2 = sum_rank_2 - (df_2.shape[0]*(df_2.shape[0]+1))/2

# imprimimos ambos
print(u_1)
print(u_2)

# Calculamos también nuestro U_statistic
U_statistic = min(u_1, u_2)

# Nuevamente para esto tenemos una tabla del U_statistic dependiendo si tenemos dos colas o 1 cola: Para este caso es 17...
# Como el u_statistic no es menor al estadístico crítico, no podemos rechazar la hipótesis nula de que ambas muestras tienen la misma mediana. Por lo tanto, ambas
# muestras provienen de una población equidistribuida (no podemos rechazar esta afirmación)

71.0
100.0
35.0
45.0


In [19]:
# U-test (p-value y tamaño de efecto)
# ==============================================================================
import pingouin as pg
pg.mwu(x=muestra_1['muestra_1'], y=muestra_2['muestra_2'], alternative='two-sided')

,U-val,alternative,p-val,RBC,CLES
MWU,35.0,two-sided,0.686662,-0.125,0.4375


## Categorical data

Generalmente en todos los datos que analizamos anteriormente teníamos una variable numérica que la relacionamos con algunos grupos categóricos y comparar medias o medianas. Digamos por ejemplo, el promedio de notas que se sacan los estudiantes respecto a dos cursos distintos, el curso A o el curso B.

Otra forma que podríamos tener, que posiblemente sea la más conocida sea relacionar dos variables numéricas. En este caso los análisis que vimos son análisis de correlación o análisis de regresiones en donde queremos ver como es la dinámica de dos variable numéricas posiblemente continuas.

Pero podemos tener un ejemplo adicional en donde tenemos dos variables categoricas que queremos analizar. Por ejemplo, podemos dos grupos el A y el B, un grupo de test y un grupo de control, donde a uno se le muestra un componente de una aplicación para suscribirse a un programa y a otro no y queremos analizar si el usuarios se termina suscribiendo o no. Por ende, tenemos 2 categorías que queres relacionar. Este es el caso que vamos a ver a continuación....

### One sample Z-test (for proportions)

En este caso estamos hablando de proporciones sobre conjuntos categoricos (algo que sucede vs algo que no sucede). Los supuestos que tenemos que tener en cuenta en este caso son que:

* Las muestras son elegidas al azar.
* Las observaciones son independientes.
* Tenemos que tener un correcto tamaño muestral: Este supuesto nos ayuda a llevar el caso binomial del test de hipotesis a un caso normal.

El estadístico de prueba \( Z \) para una prueba de una proporción muestral se calcula como:

$$
Z = \frac{\hat{p} - p_0}{\sqrt{\frac{p_0 (1 - p_0)}{n}}}
$$

donde:

- hat{p} es la proporción observada en la muestra.
- p_0 es la proporción esperada bajo la hipótesis nula.
- n es el tamaño de la muestra.
- El denominador es el error estandar para proporciones.

Tengan en cuenta que hay formulas que calculan el error estandar del estadístico con p_hat en lugar de con p_0 y esto depende del conocimiento que tengamos de la población total. Es la clasica diferencia entre utilizar un T-test donde no se conoce el desvio standar de la población o el Z test donde se da por conocido el desvio estandar de la población. En el primer caso se aproxima con el parámetro muestral que tenemos, en el segundo se utiliza el parámetro de la hipótesis nula.

Ejemplo:

Supongamos que quisieramos comparar algorítimos de recomendaciones de productos para ver si la proporción de recomendaciones de 5 estrellas es mayor en un algorítmo que en otro...

**H0: Proporcion P de 5 estrellas = 0.75 o p<= 0.75 \
Ha: Proporción P de 5 estrellas > 0.75**

(En este caso tenemos el test a una sola cola)

In [46]:
# Data de clientes
usuarios = np.arange(0,200) + 1
usuarios

#rating = np.random.randint(1, 6, size=16)
#rating

rating = np.random.choice([1, 2, 3, 4, 5], size=200, p=[0.05, 0.025, 0.05, 0.075, 0.8])
rating

array([5, 5, 5, 1, 5, 5, 5, 4, 5, 5, 5, 3, 5, 4, 5, 5, 4, 5, 5, 5, 5, 5,
       2, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5,
       4, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 4, 5, 5, 5,
       5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 4, 5, 5, 5, 5, 5, 5, 3,
       5, 4, 5, 5, 5, 5, 5, 1, 4, 5, 5, 3, 5, 3, 5, 5, 5, 3, 5, 5, 5, 5,
       5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 1, 4, 5, 5, 5, 5, 5, 5, 5, 5,
       5, 5, 5, 5, 5, 5, 5, 3, 4, 3, 5, 5, 5, 4, 2, 5, 5, 5, 5, 4, 5, 5,
       3, 5, 5, 5, 5, 5, 5, 5, 5, 1, 5, 5, 4, 5, 5, 4, 5, 5, 5, 5, 5, 4,
       5, 5, 5, 5, 5, 5, 4, 5, 5, 5, 5, 1, 5, 5, 5, 5, 5, 5, 3, 5, 5, 5,
       5, 5])

In [47]:
len(usuarios)

200

In [53]:
(rating == 5).sum()

np.int64(168)

In [61]:
# Definimos los parámetros del modelo:
alpha = 0.05
n = len(usuarios)
p_hat = (rating == 5).sum() / n
p_0 = 0.75

# Definimos el DEnominador del estadístico
SE = np.sqrt(p_0*(1-p_0)/n)

# Definimos el estadístico
Z = (p_hat - p_0) /SE
print(Z)

2.939387691339813


In [64]:
# En este caso como el test es a una única cola no tenemos que divir por dos el alpha
Z_critico = st.norm.ppf(1-alpha)

p_value = 1 - st.norm.cdf(Z)
p_value_critico = 1 - st.norm.cdf(Z_critico)
print(p_value)
print(alpha)
print(Z)
print(Z_critico)

0.001644307151923985
0.05
2.939387691339813
1.6448536269514722


Conclusión: En este caso vemos que el p_value cae en la zona de rechazo del test de hipótesis, es decir tiene un p_value mucho menor al nivel de confianza del test de error de tipo 1 que nosotros le pedimos al modelo. Entonces tenemos evidencias suficientes para poder rechazar la hipótesis nula y acercanos a una hipótesis de que los nuevos ratings tienen mejores calificaciones que las ya evidenciadas

En los casos el muestreo aleatorio no nos arroje un p-value para rechazar al hipótesis nula aunque veamos que la proporción es diferente (Ej: 0.8 vs 0.75) puede ser porque la diferencia no es del todo significativa respecto al sample size total.

### Chi-squared test (independence)

Se utiliza cuando queremos ver si existe alguna relación entre dos variables categoricas. Los supuestos son:

* Las muestras son elegidas al azar.
* Las observaciones son independientes.
* Las frecuencias esperadas de cada grupo tienen que ser como mínimo 5. Esto se lo ve mejor en la tabla de contingencias.

El estadístico de prueba \( Z \) para una prueba de una proporción muestral se calcula como:

$$
\chi^2 = \sum_{i=1}^{r} \sum_{j=1}^{c} \frac{(O_{ij} - E_{ij})^2}{E_{ij}}
$$

Donde:
- O es el valor observado de una determinada componente
- E es el valor esperado de una determinada componente
- R y C son el numero de filas y de culumnas y hacen a los grados de libertad (r-1)(c-1)

Supongamos que una compañia esta testeando dos juegos en una app y quiere ser saber si el nivel de eduación tiene alguna relación con estos juegos. Queremos hacer esto con almenos un 5% de confianza. En este caso podemos plantear el problema como que:

**HO: La aplicación que se utiliza es independiente del nivel de educación /
HA: La aplicación que se utiliza depende del nivel de eduación**

Conociendo la distribución de Chi-cuadrado sabemos que al igual que la distribución de T-Student depende de los grados de libertad de la distribución. En este caso a medida que aumentan los grados de libertad de la distribución, la misma se vuelve más simétrica

In [73]:
# Si por ejemplo las columnas nos dicen la categoria de educación (A y B) y las filas son las aplicaciones (P1 Y P2)
crosstab_matrix = np.array([[6,21],[18,15]])
crosstab_matrix

array([[ 6, 21],
       [18, 15]])

In [68]:
# Sumarizo ahora las filas 
sum_filas = crosstab_matrix.sum(axis=1)
sum_cols = crosstab_matrix.sum(axis=0)

In [78]:
# Los valores esperados se calculan de tal forma porque... supongamos que queremos saber la probabilidad que elegir una persona de educación A
print(sum_cols[0]/crosstab_matrix.sum())

# Haciendo lo mismo con la educación B vamos a llegar a al 60% pero estas proporciones se tienen que mantener para las aplicaciones entonces las distribucimos..
print((sum_cols[0]/crosstab_matrix.sum())*sum_filas[0])

0.4
10.8


In [72]:
# Calculamos los valores esperados de cada componente
E1 = sum_filas[0]*sum_cols[0]/crosstab_matrix.sum()  
E2 = sum_filas[0]*sum_cols[1]/crosstab_matrix.sum()  
E3 = sum_filas[1]*sum_cols[0]/crosstab_matrix.sum()  
E4 = sum_filas[1]*sum_cols[1]/crosstab_matrix.sum()  
E4

np.float64(19.8)

In [83]:
# Calculamos el estadístico
x2_1 = ((crosstab_matrix[0][0] - E1)**2)/E1
x2_2 = ((crosstab_matrix[0][1] - E2)**2)/E2
x2_3 = ((crosstab_matrix[1][0] - E3)**2)/E3
x2_4 = ((crosstab_matrix[1][1] - E4)**2)/E4

# Sumarizamos:
X_stat = x2_1 + x2_2 + x2_3 + x2_4
print(X_stat)

6.464646464646466


In [85]:
# Estadístico crítico (cuantil del 95%)
alpha = 0.05
df = (2-1)*(2-1)
chi2_critico = chi2.ppf(1 - alpha, df)
print(chi2_critico)

3.841458820694124


In [84]:
# Calculamos el valor crítico
from scipy.stats import chi2

# Valor-p (cola derecha)
p_value = chi2.sf(X_stat, 1)
p_value

np.float64(0.01100415317686874)

Como el p-value es menor al 5% podemos rechazar la hipótesis nula de que ambas muestras son independientes y podemos sospechar que existe cierta relación entre ambas muestras.